# Racial Disparities in Policing and Prosecution
### A Case Study in Orange County

In [1]:
# Import statements
import pandas as pd
import numpy as np

# Display all columns
pd.set_option('display.max_columns', None)

## Import the policing and prosecution data

In [2]:
# Import 2021-2023 cleaned RIPA policing dataset for Orange County directly from my GitHub
# See "RIPA Cleaning.ipynb" for details on how these were generated from the raw data
ripa_orange_2021 = "https://raw.githubusercontent.com/laurenbchu/honors-thesis/main/data/cleaned_ripa_orange_2021.csv"
ripa_orange_2022 = "https://raw.githubusercontent.com/laurenbchu/honors-thesis/main/data/cleaned_ripa_orange_2022.csv"
ripa_orange_2023 = "https://raw.githubusercontent.com/laurenbchu/honors-thesis/main/data/cleaned_ripa_orange_2023.csv"

ripa_21 = pd.read_csv(ripa_orange_2021)
ripa_22 = pd.read_csv(ripa_orange_2022)
ripa_23 = pd.read_csv(ripa_orange_2023)

In [3]:
# Adding in a YEAR column for ease of use later
ripa_21['YEAR'] = 2021
ripa_22['YEAR'] = 2022
ripa_23['YEAR'] = 2023

In [4]:
# Concatenate all three RIPA datasets together to match the RJA prosecution dataset
policing = pd.concat([ripa_21, ripa_22, ripa_23], axis=0)

# Mapping the codes for race (RAE_FULL) from numbers to race strings
races = {
    1: "Asian",
    2: "Black/African American",
    3: "Hispanic/Latino",
    4: "Middle Eastern/South Asian",
    5: "Native American",
    6: "Pacific Islander",
    7: "White",
    8: "Multiracial"
}

policing['RAE_FULL'] = policing['RAE_FULL'].map(races)

In [5]:
# Import the cleaned ACLU prosecutorial data for Orange County 2021-2023 directly from my GitHub
cleaned_orange_aclu_2021_2023 = "https://raw.githubusercontent.com/laurenbchu/honors-thesis/main/data/cleaned_orange_aclu_2021_2023.csv"
prosecution = pd.read_csv(cleaned_orange_aclu_2021_2023, low_memory=False)

In [6]:
# Adding in a year column for ease of use later
prosecution['filed_date'] = pd.to_datetime(prosecution["filed_date"])
prosecution['year'] = prosecution['filed_date'].dt.year

## Importing Census Data

In [101]:
# Imports the census data
acs_orange_2021 = "https://raw.githubusercontent.com/laurenbchu/honors-thesis/main/data/acs_5year_orangecounty_2021.csv"
acs_orange_2022 = "https://raw.githubusercontent.com/laurenbchu/honors-thesis/main/data/acs_5year_orangecounty_2022.csv"
acs_orange_2023 = "https://raw.githubusercontent.com/laurenbchu/honors-thesis/main/data/acs_5year_orangecounty_2023.csv"

acs_21 = pd.read_csv(acs_orange_2021)
acs_22 = pd.read_csv(acs_orange_2022)
acs_23 = pd.read_csv(acs_orange_2023)

In [102]:
# Select only the relevant rows of the ACS Census data, which is "HISPANIC OR LATINO AND RACE"
acs = acs_21.iloc[[74, 80, 81, 82, 83, 84, 85, 86]].reset_index()
acs = acs.drop(columns=['Orange County, California!!Percent', 'Orange County, California!!Percent Margin of Error', 'index'])
acs = acs.rename(columns={
    'Label (Grouping)': 'Group', 
    'Orange County, California!!Estimate': '2021 Population Estimate', 
    'Orange County, California!!Margin of Error': '2021 Margin of Error'
})

In [103]:
# Add on the 2022 data
acs_22 = acs_22.iloc[[76, 82, 83, 84, 85, 86, 87, 88]].reset_index()
acs['2022 Population Estimate'] = acs_22['Orange County, California!!Estimate']
acs['2022 Margin of Error'] = acs_22['Orange County, California!!Margin of Error']

In [104]:
# Add on the 2023 data
acs_23 = acs_23.iloc[[79, 85, 86, 87, 88, 89, 90, 91]].reset_index()
acs['2023 Population Estimate'] = acs_23['Orange County, California!!Estimate']
acs['2023 Margin of Error'] = acs_23['Orange County, California!!Margin of Error']

In [105]:
# Converts the estimates into integers

acs['2021 Population Estimate'] = acs['2021 Population Estimate'].str.replace(",", "")
acs['2021 Population Estimate'] = pd.to_numeric(acs['2021 Population Estimate'])

acs['2022 Population Estimate'] = acs['2022 Population Estimate'].str.replace(",", "")
acs['2022 Population Estimate'] = pd.to_numeric(acs['2022 Population Estimate'])

acs['2023 Population Estimate'] = acs['2023 Population Estimate'].str.replace(",", "")
acs['2023 Population Estimate'] = pd.to_numeric(acs['2023 Population Estimate'])

# Fix weird naming issue

acs["Group"] = [
    "Hispanic or Latino (of any race)",
    "White alone",
    "Black or African American alone",
    "American Indian and Alaska Native alone",
    "Asian alone",
    "Native Hawaiian and Other Pacific Islander alone",
    "Some other race alone",
    "Two or more races"
]

## Data Summary

In [84]:
# Policing (RIPA) data

print(f'The RIPA Orange County policing data has', policing.shape[0], f'rows and', policing.shape[1], f'columns')
print(f'\nHere are the number of rows broken down by year:', policing.groupby("YEAR").size())
print(f'\nHere are the number of rows broken down by race:', policing["RAE_FULL"].value_counts())
print(f'\nThe NAs per column in descending order are as follows:\n', policing.isna().sum().sort_values(ascending=False))

The RIPA Orange County policing data has 447512 rows and 18 columns

Here are the number of rows broken down by year: YEAR
2021     90714
2022    189834
2023    166964
dtype: int64

Here are the number of rows broken down by race: RAE_FULL
Hispanic/Latino               192344
White                         156527
Asian                          43551
Black/African American         22186
Middle Eastern/South Asian     20763
Multiracial                     6040
Native American                 3690
Pacific Islander                2411
Name: count, dtype: int64

The NAs per column in descending order are as follows:
 RAE_FULL                  0
REASON_FOR_STOP           0
ADS_SEARCH_PERSON         0
ADS_SEARCH_PROPERTY       0
CED_NONE_CONTRABAND       0
CED_FIREARM               0
CED_AMMUNITION            0
CED_WEAPON                0
CED_DRUGS                 0
CED_ALCOHOL               0
CED_MONEY                 0
CED_DRUG_PARAPHERNALIA    0
CED_STOLEN_PROP           0
CED_ELECT_DEVICE 

In [106]:
# Prosecution (RJA) data

print(f'The RJA Orange County prosecution data has', prosecution.shape[0], f'rows and', prosecution.shape[1], f'columns')
print(f'\nHere are the number of rows broken down by year:', prosecution.groupby("year").size())
print(f'\nHere are the number of rows broken down by race:', prosecution["canonical_race"].value_counts())
print(f'\nThe NAs per column in descending order are as follows:\n', prosecution.isna().sum().sort_values(ascending=False))

The RJA Orange County prosecution data has 259118 rows and 30 columns

Here are the number of rows broken down by year: year
2021    102847
2022    102820
2023     53451
dtype: int64

Here are the number of rows broken down by race: canonical_race
Latinx    132414
White      88792
Black      15177
Asian      11808
Other      10927
Name: count, dtype: int64

The NAs per column in descending order are as follows:
 statute_level              44894
sentenced_date             35791
category                      72
disposition_description       30
was_convicted                 30
was_filed_by_da                8
referral_date                  0
filed_date                     0
county_id                      0
source_case_id                 0
source_defendant_id            0
source_incident_id             0
disposition_date               0
source_docket_id               0
statute                        0
type                           0
disposition_id                 0
race_id                

## Policing Rates

In [114]:
# Create a column to indicate if any search occurred 
policing["SEARCHED"] = (
    (policing["ADS_SEARCH_PERSON"] == 1) | 
    (policing["ADS_SEARCH_PROPERTY"] == 1)
).astype(int)

# Calculate the search rate by race by year
search_rate_2021 = policing[policing['YEAR'] == 2021].groupby("RAE_FULL")["SEARCHED"].mean()
search_rate_2022 = policing[policing['YEAR'] == 2022].groupby("RAE_FULL")["SEARCHED"].mean()
search_rate_2023 = policing[policing['YEAR'] == 2023].groupby("RAE_FULL")["SEARCHED"].mean()

In [115]:
# Create a column to indicate if any hit occurred 
contraband = [
    "CED_FIREARM", 
    "CED_AMMUNITION", 
    "CED_WEAPON", 
    "CED_DRUGS", 
    "CED_ALCOHOL", 
    "CED_MONEY", 
    "CED_DRUG_PARAPHERNALIA", 
    "CED_STOLEN_PROP", 
    "CED_ELECT_DEVICE", 
    "CED_OTHER_CONTRABAND"
]

policing["HIT"] = (policing[contraband].sum(axis=1) > 0)

# Calculate the contraband hit rate by race by year
hit_rate_2021 = policing[policing['YEAR'] == 2021].groupby("RAE_FULL")["HIT"].mean()
hit_rate_2022 = policing[policing['YEAR'] == 2022].groupby("RAE_FULL")["HIT"].mean()
hit_rate_2023 = policing[policing['YEAR'] == 2023].groupby("RAE_FULL")["HIT"].mean()

In [119]:
# Create one table

policing_tbl = pd.DataFrame({
    "2021 Search Rate": search_rate_2021,
    "2022 Search Rate": search_rate_2022,
    "2023 Search Rate": search_rate_2023,
    "2021 Hit Rate": hit_rate_2021,
    "2022 Hit Rate": hit_rate_2022,
    "2023 Hit Rate": hit_rate_2021,
    "2021 Search Counts": policing[policing['YEAR'] == 2021].groupby("RAE_FULL")["SEARCHED"].count(),
    "2022 Search Counts": policing[policing['YEAR'] == 2022].groupby("RAE_FULL")["SEARCHED"].count(),
    "2023 Search Counts": policing[policing['YEAR'] == 2023].groupby("RAE_FULL")["SEARCHED"].count(),
    "2021 Hit Counts": policing[policing['YEAR'] == 2021].groupby("RAE_FULL")["HIT"].count(),
    "2022 Hit Counts": policing[policing['YEAR'] == 2022].groupby("RAE_FULL")["HIT"].count(),
    "2023 Hit Counts": policing[policing['YEAR'] == 2023].groupby("RAE_FULL")["HIT"].count()
}).rename_axis('Perceived Race')

# Add population counts to table
order = [
    "Asian alone",
    "Black or African American alone",
    "Hispanic or Latino (of any race)",
    "Middle Eastern/South Asian",
    "Two or more races",
    "American Indian and Alaska Native alone",
    "Native Hawaiian and Other Pacific Islander alone",
    "White alone"
]

simplified_acs = acs.copy().drop(columns=["2021 Margin of Error", "2022 Margin of Error", "2023 Margin of Error"])
simplified_acs = simplified_acs.set_index("Group").reindex(order).reset_index()

policing_tbl["2021 Population Estimate"] = simplified_acs["2021 Population Estimate"].values
policing_tbl["2022 Population Estimate"] = simplified_acs["2022 Population Estimate"].values
policing_tbl["2023 Population Estimate"] = simplified_acs["2023 Population Estimate"].values

policing_tbl

,2021 Search Rate,2022 Search Rate,2023 Search Rate,2021 Hit Rate,2022 Hit Rate,2023 Hit Rate,2021 Search Counts,2022 Search Counts,2023 Search Counts,2021 Hit Counts,2022 Hit Counts,2023 Hit Counts,2021 Population Estimate,2022 Population Estimate,2023 Population Estimate
Perceived Race,,,,,,,,,,,,,,,
Asian,0.111322,0.063551,0.063981,0.053381,0.032497,0.053381,5264,20094,18193,5264,20094,18193,672015.0,682736.0,687982.0
Black/African American,0.212733,0.217693,0.219938,0.094720,0.103983,0.094720,3864,9665,8657,3864,9665,8657,50161.0,48898.0,48224.0
Hispanic/Latino,0.220207,0.200304,0.212347,0.114177,0.102534,0.114177,44063,78306,69975,44063,78306,69975,1083093.0,1077367.0,1080480.0
Middle Eastern/South Asian,0.076682,0.055274,0.055184,0.031927,0.021614,0.031927,3508,8883,8372,3508,8883,8372,NaN,NaN,NaN
Multiracial,0.187500,0.120293,0.130754,0.087500,0.069104,0.087500,720,2735,2585,720,2735,2585,112689.0,121149.0,129850.0
Native American,0.005240,0.035147,0.191176,0.003368,0.019274,0.003368,2672,882,136,2672,882,136,4423.0,3771.0,3634.0
Pacific Islander,0.173362,0.177043,0.162637,0.103594,0.078794,0.103594,473,1028,910,473,1028,910,8073.0,8655.0,8779.0
White,0.170580,0.165253,0.163788,0.095854,0.092056,0.095854,30150,68241,58136,30150,68241,58136,1242725.0,1221176.0,1191304.0


In [113]:
# Create a table of per-capita search and hit rates
percapita_policing_tbl = pd.DataFrame({})
percapita_policing_tbl["2021 Per-Capita Search Rate"] = policing_tbl["2021 Search Counts"] / policing_tbl["2021 Population Estimate"]
percapita_policing_tbl["2022 Per-Capita Search Rate"] = policing_tbl["2022 Search Counts"] / policing_tbl["2022 Population Estimate"]
percapita_policing_tbl["2023 Per-Capita Search Rate"] = policing_tbl["2023 Search Counts"] / policing_tbl["2023 Population Estimate"]
percapita_policing_tbl

,2021 Per-Capita Search Rate,2022 Per-Capita Search Rate,2023 Per-Capita Search Rate
Perceived Race,,,
Asian,0.007833,0.029432,0.026444
Black/African American,0.077032,0.197656,0.179516
Hispanic/Latino,0.040683,0.072683,0.064763
Middle Eastern/South Asian,NaN,NaN,NaN
Multiracial,0.006389,0.022576,0.019908
Native American,0.604115,0.233890,0.037424
Pacific Islander,0.058590,0.118775,0.103656
White,0.024261,0.055881,0.048800


## Prosecution Rates

In [16]:
prosecution["was_referred_by_lea"].value_counts()

was_referred_by_lea
True     214224
False     44894
Name: count, dtype: int64

In [120]:
prosecution["was_filed_by_da"].value_counts()

# Because of this result, we cannot compute a disparity rate because there's too little False values

was_filed_by_da
True     259068
False        42
Name: count, dtype: int64

In [18]:
# Calculate conviction rate per race per year
conviction_rates = prosecution.groupby(["canonical_race", "year"])["was_convicted"].agg(["sum", "count"]).reset_index()
conviction_rates.rename(columns={"sum": "number_convicted", "count": "total_cases"}, inplace=True)
conviction_rates["conviction_rate"] = conviction_rates["number_convicted"] / conviction_rates["total_cases"]

conviction_rates

,canonical_race,year,number_convicted,total_cases,conviction_rate
0,Asian,2021,3233,4578,0.706204
1,Asian,2022,3051,4228,0.721618
2,Asian,2023,2256,3002,0.751499
3,Black,2021,3829,5250,0.729333
4,Black,2022,4828,6609,0.730519
5,Black,2023,2208,3312,0.666667
6,Latinx,2021,38969,50491,0.771801
7,Latinx,2022,42074,54525,0.771646
8,Latinx,2023,20381,27383,0.744294
9,Other,2021,2871,4653,0.617021


In [19]:
# Calculate enhancement rate, per race per year
enhancement_rates = prosecution.groupby(["canonical_race", "year"])["is_enhancement_charge"].agg(["sum", "count"]).reset_index()
enhancement_rates.rename(columns={"sum": "number_enhancements", "count": "total_cases"}, inplace=True)
enhancement_rates["enhancement_rate"] = enhancement_rates["number_enhancements"] / enhancement_rates["total_cases"]

enhancement_rates

,canonical_race,year,number_enhancements,total_cases,enhancement_rate
0,Asian,2021,494,4578,0.107907
1,Asian,2022,422,4228,0.099811
2,Asian,2023,246,3002,0.081945
3,Black,2021,395,5256,0.075152
4,Black,2022,426,6609,0.064458
5,Black,2023,151,3312,0.045592
6,Latinx,2021,4672,50505,0.092506
7,Latinx,2022,4556,54526,0.083556
8,Latinx,2023,2085,27383,0.076142
9,Other,2021,681,4653,0.146357


In [20]:
# Table with convictions and enhancements
merged = conviction_rates.merge(
    enhancement_rates[["canonical_race", "year", "number_enhancements", "enhancement_rate"]],
    on=["canonical_race", "year"],
    how="left"
)    

# Convert rates to percentages for readability
merged["conviction_rate"] = pd.to_numeric(merged["conviction_rate"], errors="coerce")
merged["enhancement_rate"] = pd.to_numeric(merged["enhancement_rate"], errors="coerce")
merged["conviction_rate"] = merged["conviction_rate"] * 100
merged["enhancement_rate"] = merged["enhancement_rate"] * 100

# Round to nearest thousandth
merged["conviction_rate"] = merged["conviction_rate"].round(2)
merged["enhancement_rate"] = merged["enhancement_rate"].round(2)

# Rename from rate to percentage
merged = merged.rename(columns={
        "conviction_rate": "conviction_percentage",
        "enhancement_rate": "enhancement_percentage"
})

# Only 2023
merged[merged["year"] == 2023]

,canonical_race,year,number_convicted,total_cases,conviction_percentage,number_enhancements,enhancement_percentage
2,Asian,2023,2256,3002,75.15,246,8.19
5,Black,2023,2208,3312,66.67,151,4.56
8,Latinx,2023,20381,27383,74.43,2085,7.61
11,Other,2023,1295,2003,64.65,165,8.24
14,White,2023,13405,17750,75.52,1138,6.41


In [21]:
# Calculate distribution of types of charges
prosecution["statute_level"].value_counts()

charge_distribution = prosecution.groupby(["canonical_race", "year", "statute_level"]).size().reset_index(name="number_of_cases")
charge_distribution["percent_of_cases"] = charge_distribution.groupby(["canonical_race", "year"])["number_of_cases"].transform(lambda x: x / x.sum())

# Only 2023
charge_distribution[charge_distribution["year"] == 2023].head(9)

,canonical_race,year,statute_level,number_of_cases,percent_of_cases
6,Asian,2023,Felony,876,0.397459
7,Asian,2023,Infraction,3,0.001361
8,Asian,2023,Misdemeanor,1325,0.601180
13,Black,2023,Felony,1121,0.432986
14,Black,2023,Infraction,1,0.000386
15,Black,2023,Misdemeanor,1467,0.566628
22,Latinx,2023,Felony,7188,0.332978
23,Latinx,2023,Infraction,24,0.001112
24,Latinx,2023,Misdemeanor,14375,0.665910
